## California Housing Project

This lab shows the importance of location in the pricing of housing in California, showing that location along can account for 75% of the variation in housing prices. 
- The **first model** considers the house's simple geographic region and accounts for **20%** of the variation in housing price.
- The **second model** considers the house's distance to large cities and accounts for **25%** of variation.
- The **final model** considers the price of nearby homes and accounts for **75%** of the variation.

## EDA
